In [ ]:
# default_exp clone_analysis

In [ ]:
#export
from functools import reduce

import numpy as np
import pandas as pd

# Clone Analysis Functions

In [ ]:
# export
def combine_agg_functions(additional_agg_functions):
    if additional_agg_functions is None:
        additional_agg_functions = {}

    agg_functions = {"label": "count", "area_um2": [np.mean, np.std]}
    return {**agg_functions, **additional_agg_functions}

In [ ]:
# export
def individual_filter_condition(
    df, filtered_col_name: str, query: str, clone_channel: str, agg_functions
):
    temp_df = (
        df.query(query).groupby(["int_img", clone_channel]).agg(agg_functions)
    ).copy()

    temp_df.columns = pd.MultiIndex.from_tuples(
        [(filtered_col_name,) + a for a in temp_df.columns]
    )
    return temp_df

In [ ]:
# export
def query_df_groupby_by_clone_channel(
    df,
    queries: dict,
    clone_channel: str = "C1",
    additional_agg_functions: dict = None,
):
    """additional agg_functions could be something like:
    additional_agg_functions = {"mean_intensity": [np.mean, np.std]}"""

    agg_functions = combine_agg_functions(additional_agg_functions)
    df = df.reset_index()

    l = list()
    for key, query in queries.items():
        l.append(
            individual_filter_condition(df, key, query, clone_channel, agg_functions)
        )

    return reduce(
        lambda df_left, df_right: pd.merge(
            df_left, df_right, left_index=True, right_index=True
        ),
        l,
    )

In [ ]:
from py_clone_detective.clone_counters import LazyCloneCounter
from dask.distributed import Client

In [ ]:
c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:49761,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:49770,Total threads: 2
Dashboard: http://127.0.0.1:49774/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:49765,


## Example using LazyCloneCounter with measure_overlap

In [ ]:
bar = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

bar.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

bar.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
bar.make_measurements(extra_properties=["convex_area"],)
bar.measure_overlap()
bar.combine_C0_overlaps_and_measurements()

C1   C2  C3 seg_ch int_img_ch  int_img  area  \
img_name C0_labels                                                 
a1g01p1  1           0    0   0     C0         C0  a1g01p1   353   
         2           0    0   0     C0         C0  a1g01p1   613   
         3           0    0   0     C0         C0  a1g01p1   275   
         4           0    3   0     C0         C0  a1g01p1   700   
         5           0    0   0     C0         C0  a1g01p1    33   
...                 ..  ...  ..    ...        ...      ...   ...   
a2g13p3  249         0  150   0     C0         C0  a2g13p3   712   
         250         0  146   0     C0         C0  a2g13p3   308   
         251         0  148   0     C0         C0  a2g13p3   678   
         252         0    0   0     C0         C0  a2g13p3   672   
         253         0    0   0     C0         C0  a2g13p3   212   

                    mean_intensity  centroid-0  centroid-1  convex_area  \
img_name C0_labels                                                        
a1g01p1  1             1143.484419    5.025496   97.269122          366   
         2             2521.355628    8.014682  222.654160          640   
         3             2570.625455    5.941818  383.338182          288   
         4             2873.265714   14.550000  519.455714          724   
         5             2838.606061    0.848485  579.636364           35   
...                            ...         ...         ...          ...   
a2g13p3  249           1801.213483  792.443820  551.751404          739   
         250           1980.500000  793.714286  459.652597          318   
         251           2612.631268  799.069322  500.057522          696   
         252           1393.748512  798.055060  786.084821          689   
         253           2926.056604  805.608491  636.231132          219   

                     area_um2  convex_area_um2  
img_name C0_labels                              
a1g01p1  1          26.695625        27.678750  
         2          46.358125        48.400000  
         3          20.796875        21.780000  
         4          52.937500        54.752500  
         5           2.495625         2.646875  
...                       ...              ...  
a2g13p3  249        53.845000        55.886875  
         250        23.292500        24.048750  
         251        51.273750        52.635000  
         252        50.820000        52.105625  
         253        16.032500        16.561875  

[21336 rows x 13 columns]

## Example using LazyCloneCounter with add_clones_and_neighbouring_labels

In [ ]:
foo = PersistentCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

In [ ]:
foo.add_segmentations(
    morphology.remove_small_objects,
    ad_func_kwargs={"min_size": 49},
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
)